In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo
import pickle
from sklearn.preprocessing import StandardScaler


In [2]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'

# Open the .pkl file
with open(DATOS_DIR+'/scalers.pkl', 'rb') as f:
        scalers = pickle.load(f)
   
# Function to center, scale, and return a series
def minmax_scale_group(group):
    centered_values = group
    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(centered_values.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_minmax_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values
    return pd.Series(original_values, index=group.index, name=group_name)



In [3]:

# Leer datos
df_final = pd.read_parquet(DATOS_DIR+'/FE_dataset-CARLA.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)


In [4]:

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2020-01-01']



#Filtro test
#df_final = df_final[df_final['product_id'] < 20013]

In [5]:
df_final['tn_2'].fillna(0, inplace=True)


C:\Users\Admin\AppData\Local\Temp\ipykernel_31644\3120981005.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['tn_2'].fillna(0, inplace=True)


In [6]:
# Correr Modelo



params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        #'seed': 113,
        #'learning_rate': 0.2,
        #'bagging_fraction': 0.85,
        #'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0

for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    model, average_metric = labo.train_lightgbm_model(df_producto,params,metric='rmse')
    print("Overall rmse metric: ", average_metric)
    # Predict values for the entire dataset using the trained models
    # Prepare last data points for prediction
    last_data_points = df_producto[df_producto.index == df_producto.index.max()].copy()
    last_data_points.drop(columns=['tn_2'], inplace=True)
    # Predict the next month's value using the trained model
    predictions = labo.predict_next_month(model, last_data_points)
    preds = predictions.groupby('product_id')['tn_2'].transform(inverse_minmax_scale_group)
    predictions['tn'] = preds
    predictions.drop(columns=['tn_2'], inplace=True)
    predictions = predictions.reset_index()
    predictions =  predictions.groupby('product_id')['tn'].sum()
    predictions.columns = ['product_id', 'tn']
    predictions_all = pd.concat([predictions_all, predictions])
    print(predictions_all[-1:])


Fitting and predicting for product_id: 20524
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[41]	validation's rmse: 0.149289
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[38]	validation's rmse: 0.194381
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[48]	validation's rmse: 0.330124
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[36]	validation's rmse: 0.0758901
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[39]	validation's rmse: 0.0389455
Overall rmse metric:  0.0015167549743498568
              tn
20524  53.718196
Fitting and predicting for product_id: 20311
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[23]	validation's rmse: 0.307553


C:\Users\Admin\AppData\Local\Temp\ipykernel_31644\66188192.py:46: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_31644\66188192.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[27]	validation's rmse: 0.248834
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[62]	validation's rmse: 0.301178
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[94]	validation's rmse: 0.120589
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[26]	validation's rmse: 0.198419
Overall rmse metric:  0.014541672839280515
               tn
20311  153.615219
Fitting and predicting for product_id: 20654
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[34]	validation's rmse: 0.0275718
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[80]	validation's rmse: 0.0125881
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[222

In [7]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all.to_csv(DATOS_DIR+'/pred/0017-prediccion-rmse_scaled_CORRECT2-product_id.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)


Overall custom metric:  3.951873967190977
